In [46]:
import os
import random
import numpy as np
import os
import random as rn
import numpy as np
DATA_DIR = './data/'

NUM_TRAIN_SAMPLES = 40000
NUM_VAL_SAMPLES = 10000

In [47]:
from utils.mask_utils import *
from utils.visualization_utils import *
from utils.transform_utils import *
from utils.data_utils import * 
from utils.augmentation_utils import * 

In [48]:
import json

f = open (DATA_DIR + './annotations_trainval2017/annotations/instances_train2017.json', "r")
train_annotations = json.loads(f.read())


f = open (DATA_DIR + './annotations_trainval2017/annotations/instances_val2017.json', "r")
val_annotations = json.loads(f.read())

In [49]:
category_dict = {1 : 'person',
                 3 : 'vehicle',
                 4 : 'vehicle',
                 6 : 'vehicle',
                 8 : 'vehicle',
                 18 : 'animal',
                 19 : 'animal',
                 20 : 'animal',
                 21 : 'animal',
                 23 : 'animal',}

In [50]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2

train_data_useful =[]
train_data_background =[]

for ann in train_annotations['annotations']:
    
    if ann['category_id'] in category_dict:
        train_data_useful.append(ann)

    else:
        train_data_background.append(ann)


val_data_useful =[]
val_data_background =[]

for ann in val_annotations['annotations']:
    
    if ann['category_id'] in category_dict:
        val_data_useful.append(ann)

    else:
        val_data_background.append(ann)

print(['Train: ' ,len(train_data_useful), 
       'Val: ', len(val_data_useful)])

print(['Train: ' ,len(train_data_background), 
       'Val: ', len(val_data_background)])

['Train: ', 362144, 'Val: ', 15310]
['Train: ', 497857, 'Val: ', 21471]


In [51]:
def get_random_category_object(category,
                             data_useful):

    found = False

    while not found:

        r_ann = random.choice(data_useful)

        if r_ann['category_id'] in category_dict:

            if category_dict[r_ann['category_id']] == category:

                return r_ann    

def get_random_background_object(data_background):

    return random.choice(data_background)  

In [52]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2

SEED = 0

for split, num_samples_per_class, data_useful in [('train', NUM_TRAIN_SAMPLES, train_data_useful),
                                     ('val', NUM_VAL_SAMPLES, val_data_useful)]:

    for category_name in ['person', 'vehicle', 'animal']:


        total_created = 0
        directory = './sentetik_data/%s/%s/' % (split, category_name)
        if not os.path.exists(directory):
            os.makedirs(directory)

        os.environ['PYTHONHASHSEED']=str(SEED)
        np.random.seed(SEED)
        rn.seed(SEED)

        while total_created < num_samples_per_class:
        

            
            # LOAD RANDOM BACKGROUND IMAGE
            random_background_image ,background_image_shape = load_random_background_image()

            cur_added = 0 
            while cur_added < 1:

                
                SEED += 1
                ann = get_random_category_object(category_name, data_useful)

                image = cv2.imread(DATA_DIR + f"{split}2017/{split}2017/%012d.jpg" % (ann['image_id']))

                h,w = image.shape[:2]

                mask = annToMask(ann,
                    h,
                    w,)
                


                # APPLY TRANSFORM TO SEGMENTATION OBJECT
                image_new , mask_new = transform_object_and_mask(image.copy(),
                                                                mask.copy(),)

                # GET TRANSFORMED OBJECT PATH AND POSITIONS
                try:
                    patch, y, x, maximum_possible_y, maximum_possible_x = get_transformed_object_patch(image_new, 
                                                                                            mask_new,
                                                                                            background_image_shape)
                except:
                    continue

                if (x.max()-x.min()) < 40 or (y.max()-y.min())<40:
                    continue

                if (maximum_possible_x <= 30) or (maximum_possible_y <= 30):
                    continue

                # PLACE TRANSFORMED OBJECT TO BACKGROUND IMAGE
                place_transformed_object_patch(random_background_image,
                                                patch,
                                                y,
                                                x,
                                                maximum_possible_y,
                                                maximum_possible_x,)

                cur_added += 1

            k = apply_augmentation(random_background_image[:,:,::-1])
        
            cv2.imwrite(directory+ '/%s_%08d.jpg' % (category_name, total_created), k[:,:,::-1] )

            total_created += 1


KeyboardInterrupt: 

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2


SEED = 0

for split, num_samples_per_class, data_useful in [('train', NUM_TRAIN_SAMPLES, train_data_background),
                                     ('val', NUM_VAL_SAMPLES, val_data_background)]:

    for category_name in ['background']:


        total_created = 0

        directory = './sentetik_data/%s/%s/' % (split, category_name)
        if not os.path.exists(directory):
            os.makedirs(directory)

        os.environ['PYTHONHASHSEED']=str(SEED)
        np.random.seed(SEED)
        rn.seed(SEED)

        while total_created < num_samples_per_class:


            # LOAD RANDOM BACKGROUND IMAGE
            random_background_image ,background_image_shape = load_random_background_image()

            cur_added = 0 
            while cur_added < 1:
                SEED += 1

                ann = get_random_background_object(data_useful)

                image = cv2.imread(DATA_DIR + f"{split}2017/{split}2017/%012d.jpg" % (ann['image_id']))

                h,w = image.shape[:2]

                mask = annToMask(ann,
                    h,
                    w,)
                


                # APPLY TRANSFORM TO SEGMENTATION OBJECT
                image_new , mask_new = transform_object_and_mask(image.copy(),
                                                                mask.copy(),)

                # GET TRANSFORMED OBJECT PATH AND POSITIONS
                try:
                    patch, y, x, maximum_possible_y, maximum_possible_x = get_transformed_object_patch(image_new, 
                                                                                            mask_new,
                                                                                            background_image_shape)
                except:
                    continue

                if (x.max()-x.min()) < 40 or (y.max()-y.min())<40:
                    continue

                if (maximum_possible_x <= 30) or (maximum_possible_y <= 30):
                    continue

                # PLACE TRANSFORMED OBJECT TO BACKGROUND IMAGE
                place_transformed_object_patch(random_background_image,
                                                patch,
                                                y,
                                                x,
                                                maximum_possible_y,
                                                maximum_possible_x,)

                cur_added += 1

            k = apply_augmentation(random_background_image[:,:,::-1])
        
            if not os.path.exists(directory):
                os.makedirs(directory)

            cv2.imwrite(directory+ '/%s_%08d.jpg' % (category_name, total_created), k[:,:,::-1] )

            total_created += 1
